# Treinando Redes Profundas
- Consultar Tabelas 11-3 e 11-4 para sugestões de redes que podem funcionar na prática.
- O exercício no fim do capítulo aparentemente indicou que SELU com dropout não é tão bom como Batch Normalization.

## Dataset

In [13]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

## Problema de Gradientes Explosivos ou que se Anulam

### Inicialização de Xavier/Glorot ou de He
- Essas inicializações buscam garantir que a variância (distribuição normal ou uniforme) da entrada é igual à da saída, de forma que os gradientes não se anulem.
- Tomam como base o fan<sub>in</sub> e o fan<sub>out</sub>
- Se recomenda, para distribuições normais, as seguintes inicializações: Glorot, para tanh ou logística, He, para ReLU e LeCun para a SELU.
- Por padrão o Keras usa a inicialização de Glorot com distribuição uniforme

In [1]:
import tensorflow as tf
from tensorflow import keras

keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

In [3]:
he_avg_init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg',
distribution='uniform')
keras.layers.Dense(10, activation="sigmoid", kernel_initializer=he_avg_init)

### Funções de Ativação que não Saturam
- ReLU, <span style='font-style:italic;'>leaky</span> ReLU, <span style='font-style:italic;'>randomized leaky</span> ReLU (RReLU), <span style='font-style:italic;'>parametric leaky</span> ReLU (PReLU: o &alpha; é aprendido durante o treinamento), ELU (para valores negativos há a suavização por uma exponencial) e SELU (versão escalonada da ELU que resolve o problema de gradientes que se anulam se os dados de entrada tiverem distribuição normal padrão, se não houver atalhos na rede se e as demais camadas seguirem a inicialização <span style='font-family:monospace;'>lecun_normal</span>)
- Em geral: SELU (não é suavizada como a ELU) > ELU > <span style='font-style:italic;'>leaky</span> ReLU > RELU > <span style='font-style:italic;'>tanh</span>() > logística. Se a latência for muito importante, RELU pode ser mais adequada

In [6]:
import numpy as np
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Dense(10, activation="softmax")
])

### Batch Normalization
- O parâmetro <span style='font-family:monospace'>axis</span> se comporta de forma contrária à biblioteca numpy, em que o axis indica a dimensão que será colapsada.

In [7]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False), # O Bias não é necessário antes da BatchNormalization()
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_6 (Dense)              (None, 300)               235200    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
activation (Activation)      (None, 300)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)               30000     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)              

### Gradient Clipping
- Restringir o valor de uma das componentes ou a norma do gradiente. É mais necessário nas redes recorrentes, nas quais Batch Normalization não é tão efetivo.

In [2]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)
optimizer = keras.optimizers.SGD(clipnorm=1.0)
# model.compile(loss="mse", optimizer=optimizer)

## Reusando Camadas Pré-Treinadas
- Quando tarefas são similares, pode-se utilizar redes já treinadas e substituir as últimas camadas. Isso faz com que não sejam necessárias tantas amostras para o treinamento. Ex: classificação de imagens.

### Transfer Learning com Keras
- Fashion MNIST: X_train_A &#8594; todas as imagens, exceto as sandálias e camisetas (classes 5 e 6). X_train_B &#8594; apenas 200 imagens de sandálias e camisetas.

In [3]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6)
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # os índices das classes 7,8,9 devem ser movidos para 5,6,7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # classificação binária
    return ((X[~y_5_or_6], y_A), (X[y_5_or_6], y_B))

In [19]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
print(X_train_full.shape)
print(X_train_full.dtype)
X_valid, X_train, X_test = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0, X_test / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
"Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

(60000, 28, 28)
uint8


In [20]:
import numpy as np

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [7]:
print(X_train_A.shape)
print(X_train_B.shape)

(43986, 28, 28)
(200, 28, 28)


In [9]:
tf.random.set_seed(42)
np.random.seed(42)

model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation='selu'))
model_A.add(keras.layers.Dense(8, activation='softmax'))

model_A.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=['accuracy'])

history = model_A.fit(X_train_A, y_train_A, epochs=20, validation_data=(X_valid_A, y_valid_A))

model_A.save('my_model_A.h5')

Epoch 1/20
1375/1375 [==============================] - 4s 2ms/step - loss: 0.5926 - accuracy: 0.8103 - val_loss: 0.3890 - val_accuracy: 0.8677
Epoch 2/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.3523 - accuracy: 0.8786 - val_loss: 0.3290 - val_accuracy: 0.8822
Epoch 3/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.3170 - accuracy: 0.8895 - val_loss: 0.3014 - val_accuracy: 0.8989
Epoch 4/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.2972 - accuracy: 0.8974 - val_loss: 0.2892 - val_accuracy: 0.9016
Epoch 5/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2834 - accuracy: 0.9023 - val_loss: 0.2774 - val_accuracy: 0.9071
Epoch 6/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2729 - accuracy: 0.9063 - val_loss: 0.2733 - val_accuracy: 0.9071
Epoch 7/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2640 - accuracy: 0.9091 - val_loss: 0.2719 - val_accuracy:

In [10]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 46ms/step - loss: 0.9573 - accuracy: 0.4650 - val_loss: 0.6314 - val_accuracy: 0.6004
Epoch 2/20
7/7 [==============================] - 0s 17ms/step - loss: 0.5692 - accuracy: 0.7450 - val_loss: 0.4784 - val_accuracy: 0.8529
Epoch 3/20
7/7 [==============================] - 0s 18ms/step - loss: 0.4503 - accuracy: 0.8650 - val_loss: 0.4102 - val_accuracy: 0.8945
Epoch 4/20
7/7 [==============================] - 0s 20ms/step - loss: 0.3879 - accuracy: 0.8950 - val_loss: 0.3647 - val_accuracy: 0.9178
Epoch 5/20
7/7 [==============================] - 0s 20ms/step - loss: 0.3435 - accuracy: 0.9250 - val_loss: 0.3300 - val_accuracy: 0.9320
Epoch 6/20
7/7 [==============================] - 0s 22ms/step - loss: 0.3081 - accuracy: 0.9300 - val_loss: 0.3019 - val_accuracy: 0.9402
Epoch 7/20
7/7 [==============================] - 0s 18ms/step - loss: 0.2800 - accuracy: 0.9350 - val_loss: 0.2804 - val_accuracy: 0.9422
Epoch 8/20
7/7 [===========

In [12]:
my_model_A = keras.models.load_model('my_model_A.h5')
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.summary()
model_B_on_A.add(keras.layers.Dense(1, activation='sigmoid'))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 300)               235500    
_________________________________________________________________
dense_8 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_9 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_10 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_11 (Dense)             (None, 50)                2550      
Total params: 275,750
Trainable params: 275,750
Non-trainable params: 0
________________________________________________

In [13]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_A_clone.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 300)               235500    
_________________________________________________________________
dense_8 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_9 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_10 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_11 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_12 (Dense)             (None, 8)                

In [14]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 37ms/step - loss: 0.4515 - accuracy: 0.8350 - val_loss: 0.4461 - val_accuracy: 0.8174
Epoch 2/4
7/7 [==============================] - 0s 9ms/step - loss: 0.4188 - accuracy: 0.8500 - val_loss: 0.4167 - val_accuracy: 0.8306
Epoch 3/4
7/7 [==============================] - 0s 9ms/step - loss: 0.3899 - accuracy: 0.8600 - val_loss: 0.3900 - val_accuracy: 0.8448
Epoch 4/4
7/7 [==============================] - 0s 8ms/step - loss: 0.3636 - accuracy: 0.8700 - val_loss: 0.3668 - val_accuracy: 0.8540


In [15]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/16
7/7 [==============================] - 1s 32ms/step - loss: 0.2965 - accuracy: 0.9250 - val_loss: 0.2504 - val_accuracy: 0.9523
Epoch 2/16
7/7 [==============================] - 0s 9ms/step - loss: 0.2079 - accuracy: 0.9850 - val_loss: 0.1981 - val_accuracy: 0.9807
Epoch 3/16
7/7 [==============================] - 0s 9ms/step - loss: 0.1648 - accuracy: 0.9900 - val_loss: 0.1665 - val_accuracy: 0.9899
Epoch 4/16
7/7 [==============================] - 0s 9ms/step - loss: 0.1372 - accuracy: 1.0000 - val_loss: 0.1461 - val_accuracy: 0.9919
Epoch 5/16
7/7 [==============================] - 0s 13ms/step - loss: 0.1192 - accuracy: 1.0000 - val_loss: 0.1309 - val_accuracy: 0.9929
Epoch 6/16
7/7 [==============================] - 0s 9ms/step - loss: 0.1053 - accuracy: 1.0000 - val_loss: 0.1194 - val_accuracy: 0.9929
Epoch 7/16
7/7 [==============================] - 0s 10ms/step - loss: 0.0950 - accuracy: 1.0000 - val_loss: 0.1103 - val_accuracy: 0.9919
Epoch 8/16
7/7 [===============

In [21]:
model_B.evaluate(X_test_B, y_test_B)
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 1ms/step - loss: 0.0637 - accuracy: 0.9980


[0.06370459496974945, 0.9980000257492065]

In [22]:
(100 - 97.05) / (100 - 99.8)

14.749999999999805

#### Comentários
- A taxa de erro caiu quase 15 vezes, entretanto, transfer learning não funciona tão bem para redes pequenas fully-connected, que detectam padrões mais específicos. Elas são mais úteis em redes profundas convolucionais, pois elas são capazes de detectar padrões mais gerais

### Pré-Treinamento Não-Supervisionado
- Quando há poucos dados rotulados e muitos não rotulados, pode-se treinar uma GAN ou um autoencoder com os dados não rotulados.
- Feito isso, se utiliza o autoencoder ou o discriminador da GAN e se adiciona outras camadas para se fazer o treinamento supervisionado com os dados rotulados.

### Pré-Treinamento em uma Tarefa Auxiliar
- Quando há poucos dados para a tarefa específica, mas muitos dados para outras tarefas similares. Nesse caso, pode-se treinar o modelo para a tarefa similar e depois re-treinar as últimas camadas para a tarefa específica.

## Otimizadores

### Termo de Momento
- O gradiente se torna termo de aceleração, não velocidade:
<br>
<p style='font-style:italic;font-size:2rem;text-align:center;'><strong>m</strong> &larr; &beta;<strong>m</strong> - &eta;&nabla;<sub style='font-weight:bold;'>&theta;</sub>J(<strong>&theta;</strong>)</p>
<p style='font-style:italic;font-size:2rem;text-align:center;'><strong>&theta;</strong> = <strong>&theta;</strong> + <strong>m</strong></p>

In [4]:
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9) # lr = η e momentum = β 

### Gradiente Acelerado de Nesterov (NAG)
- Se calcula o gradiente no local onde o vetor de pesos está indo, acelerando a convergência.
- Geralmente o NAG se comporta melhor e é mais rápido que o método padrão
<br>
<p style='font-style:italic;font-size:2rem;text-align:center;'><strong>m</strong> &larr; &beta;<strong>m</strong> - &eta;&nabla;<sub style='font-weight:bold;'>&theta;</sub>J(<strong>&theta;</strong>+&beta;<strong>m</strong>)</p>
<p style='font-style:italic;font-size:2rem;text-align:center;'><strong>&theta;</strong> = <strong>&theta;</strong> + <strong>m</strong></p>

In [5]:
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)

### Adagrad
- Penaliza as componentes dos gradientes que já foram muito ajustadas, fazendo com que a direção de ajuste permaneça mais constante.
- Entretanto, não é recomendado para redes profundas, pois acaba zerando as componentes muito rápido
<br>
<p style='font-style:italic;font-size:2rem;text-align:center;'><strong>s</strong> &larr; <strong>s</strong> + &nabla;<sub style='font-weight:bold;'>&theta;</sub><sup>2</sup>J(<strong>&theta;</strong>)</p>
<p style='font-style:italic;font-size:2rem;text-align:center;'><strong>&theta;</strong> = <strong>&theta;</strong> - &eta;&nabla;<sub style='font-weight:bold;'>&theta;</sub>J(<strong>&theta;</strong>) / <span style="white-space: nowrap;">&radic;<span style="text-decoration:overline;">&nbsp;<strong>s</strong> + &epsilon;&nbsp;</span>
</span></p>

In [7]:
optimizer = keras.optimizers.Adagrad(learning_rate=0.001)

### Adadelta
- Conserta o algoritmo anterior, adicionando o termo de esquecimento de forma que o <strong>s</strong> não cresça de forma exagerada.
<br>
<p style='font-style:italic;font-size:2rem;text-align:center;'><strong>s</strong> &larr; &beta;<strong>s</strong> + (1 - &beta;)&nabla;<sub style='font-weight:bold;'>&theta;</sub><sup>2</sup>J(<strong>&theta;</strong>)</p>
<p style='font-style:italic;font-size:2rem;text-align:center;'><strong>&theta;</strong> = <strong>&theta;</strong> - &eta;&nabla;<sub style='font-weight:bold;'>&theta;</sub>J(<strong>&theta;</strong>) / <span style="white-space: nowrap;">&radic;<span style="text-decoration:overline;">&nbsp;<strong>s</strong> + &epsilon;&nbsp;</span>
</span></p>

In [6]:
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9) # lr = η e rho = β 

### Adam
- Combina o Adadelta com o termo de momento, mas fazendo média exponencial decadente em vez de soma exponencial decadente (a diferença é apenas de uma constante multiplicativa).
- Os termos adicionais corrigem o viés em torno de zero para as primeiras iterações.

<p style='font-style:italic;font-size:2rem;text-align:center;'><strong>m</strong> &larr; &beta;<sub>1</sub><strong>m</strong> - (1 - &beta;<sub>1</sub>)&nabla;<sub style='font-weight:bold;'>&theta;</sub>J(<strong>&theta;</strong>)</p>

<p style='font-style:italic;font-size:2rem;text-align:center;'><strong>s</strong> &larr; &beta;<sub>2</sub><strong>s</strong> + (1 - &beta;<sub>2</sub>)&nabla;<sub style='font-weight:bold;'>&theta;</sub><sup>2</sup>J(<strong>&theta;</strong>)</p>

<p style='font-style:italic;font-size:2rem;text-align:center;'><strong>m</strong> &larr; <strong>m</strong> /(1 - &beta;<sub>1</sub><sup>T</sup>)</p>

<p style='font-style:italic;font-size:2rem;text-align:center;'><strong>s</strong> &larr; <strong>s</strong> /(1 - &beta;<sub>2</sub><sup>T</sup>)</p>

<p style='font-style:italic;font-size:2rem;text-align:center;'><strong>&theta;</strong> = <strong>&theta;</strong> - &eta;<strong>m</strong> / <span style="white-space: nowrap;">&radic;<span style="text-decoration:overline;">&nbsp;<strong>s</strong> + &epsilon;&nbsp;</span>
</span></p>

In [3]:
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999) # o epsilon padrão do Keras é 1e-7

## Programação da Taxa de Aprendizado

### Power Scheduling
- Tipicamente, c = 1. A taxa cai a cada s passos: &eta;<sub>0</sub>/2, &eta;<sub>0</sub>/3, etc.
<br> 
<p style='font-style:italic;font-size:1.5em;text-align:center;'>&eta;(t) = &eta;<sub>0</sub> /(1 + t/s)<sup>c</sup></p>


In [4]:
optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4) # decay é o inverso de s

### Exponencial Scheduling
- A taxa cai 10 vezes a cada s passos.
<br> 
<p style='font-style:italic;font-size:1.5em;text-align:center;'>&eta;(t) = &eta;<sub>0</sub> 0.1<sup>t/s</sup></p>

In [5]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0*0.1**(epoch/s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.01, s=20)
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
# history = model.fit(X_train_scaled, y_train, [...], callbacks=[lr_scheduler])

# Há a opção de fazer a cada batch de cada época, consultar o código do GitHub se um dia for necessário.

### Piecewise Constant Scheduling
- Constante por partes

In [6]:
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001

### Performance Scheduling
- Mede o erro de validação a cada N passos e reduz a taxa por um fator &lambda; se o erro parar de cair

In [7]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

### 1cycle Scheduling
- Inicia com uma taxa &eta;<sub>0</sub>, sobe até &eta;<sub>1</sub> no meio do treinamento, volta até &eta;<sub>0</sub> no fim e, nas últimas épocas, diminui a taxa linearmente por várias ordens de magnitude. &eta;<sub>1</sub> seria a máxima taxa de aprendizado e &eta;<sub>0</sub> 10 vezes menor que &eta;<sub>1</sub>. O momento também seguiria uma abordagem parecida: 0,95/0,85/0,95.

In [8]:
K = keras.backend

class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                 last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, rate)
        
# n_epochs = 25
# onecycle = OneCycleScheduler(math.ceil(len(X_train) / batch_size) * n_epochs, max_rate=0.05)
# history = model.fit(X_train_scaled, y_train, epochs=n_epochs, batch_size=batch_size,
#                     validation_data=(X_valid_scaled, y_valid),
#                     callbacks=[onecycle])

## Técnicas de Regularização

### Usando as Normas L1 e L2

In [9]:
layer = keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))
# or l1(0.1) for ℓ1 regularization with a factor of 0.1
# or l1_l2(0.1, 0.01) for both ℓ1 and ℓ2 regularization, with factors 0.1 and 0.01 respectively

In [10]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense, activation="elu", kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])

### Dropout
- O neurônio tem probabilidade p de não ser usado no passo atual do treinamento: 20-30% nas redes recorrentes e 40-50% nas redes convolucionais.
- Muitos modelos estado-da-arte só usam dropout na última camada escondida. Pode-se usar dropout apenas nas 3 últimas camadas escondidas se colocar dropout em todas levar a underfitting.
- Para regularizar uma rede com ativação SELU e preservar a média e a variância das saídas, utilizar alpha dropout

In [11]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

In [14]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
n_epochs = 20
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/20
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6656 - accuracy: 0.7566 - val_loss: 0.6579 - val_accuracy: 0.8356
Epoch 2/20
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5549 - accuracy: 0.7959 - val_loss: 0.5980 - val_accuracy: 0.8424
Epoch 3/20
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5255 - accuracy: 0.8060 - val_loss: 0.5502 - val_accuracy: 0.8402
Epoch 4/20
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5049 - accuracy: 0.8123 - val_loss: 0.4672 - val_accuracy: 0.8560
Epoch 5/20
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4929 - accuracy: 0.8178 - val_loss: 0.4846 - val_accuracy: 0.8574
Epoch 6/20
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4754 - accuracy: 0.8247 - val_loss: 0.4527 - val_accuracy: 0.8630
Epoch 7/20
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4718 - accuracy: 0.8257 - val_loss: 0.4765 - val_accuracy:

### Monte Carlo (MC) Dropout
- Dá uma informação mais acurada sobre a incerteza da predição e também possui o potencial de melhorar a acurácia do modelo

In [16]:
import numpy as np
tf.random.set_seed(42)
np.random.seed(42)

y_probas = np.stack([model(X_test_scaled, training=True) for sample in range(100)]) # [100, 10000, 10]
y_proba = y_probas.mean(axis=0) # [10000, 10]
y_std = y_probas.std(axis=0)

In [17]:
np.round(model.predict(X_test_scaled[:1]), 2)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [18]:
np.round(y_probas[0:10, :1], 2)

array([[[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.51, 0.  , 0.48]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.84, 0.  , 0.15]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.13, 0.  , 0.12, 0.  , 0.75]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.99]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.03, 0.  , 0.22, 0.  , 0.75]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.03, 0.  , 0.61, 0.  , 0.36]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.06, 0.  , 0.92]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.24, 0.  , 0.74]],

       [[0.  , 0.  , 0.01, 0.  , 0.03, 0.23, 0.01, 0.11, 0.  , 0.61]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.04, 0.  , 0.96]]],
      dtype=float32)

In [19]:
np.round(y_proba[:1], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.08, 0.  , 0.16, 0.  , 0.76]],
      dtype=float32)

In [20]:
y_std = y_probas.std(axis=0)
np.round(y_std[:1], 2)

array([[0.  , 0.  , 0.  , 0.01, 0.  , 0.15, 0.  , 0.21, 0.  , 0.27]],
      dtype=float32)

#### Observação
- Caso haja outras camadas que tem comportamento diferente no treinamento e na predição, como Batch Normalization, é necessário alterar as camadas que tem dropout como está descrito abaixo.
- O modelo já poderia ser construído diretamente dessa forma, mas, caso não, faz-se a alteração do modelo e copia os pesos da rede anterior para a nova rede.

In [21]:
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [22]:
tf.random.set_seed(42)
np.random.seed(42)

mc_model = keras.models.Sequential([
    MCAlphaDropout(layer.rate) if isinstance(layer, keras.layers.AlphaDropout) else layer
    for layer in model.layers
])

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
mc_model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

mc_model.set_weights(model.get_weights())

In [23]:
np.round(np.mean([mc_model.predict(X_test_scaled[:1]) for sample in range(100)], axis=0), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.08, 0.  , 0.15, 0.  , 0.77]],
      dtype=float32)

### Regularização de Máxima Norma
- O vetor de pesos é reescalonado conforme a máxima norma definida previamente: ou seja, esse critério não faz parte da função custo.

In [3]:
layer = keras.layers.Dense(100, activation='selu', kernel_initializer='lecun_normal', 
                          kernel_constraint=keras.constraints.max_norm(1.))